# 6. POS Tagger

POS Tagging stands for Part-of-Speech tagging (aka PoS tagging or POST), also called grammatical tagging, and is the process of marking up a word in a text (corpus) as corresponding to a particular part of speech, based on both its definition and its context. More info [here](https://en.wikipedia.org/wiki/Part-of-speech_tagging).

Same concept as in NER, but with our custom data to be imported in JSON format, and with custom labels (not NER, but other strings to classify our words in).

## Setup

In [1]:
! pip install --q transformers torch

In [2]:
import nltk
from nltk.corpus import brown

import json

from datasets import load_dataset

from transformers import AutoTokenizer, DataCollatorForTokenClassification, AutoModelForTokenClassification, TrainingArguments, Trainer, pipeline

## Data Load

In [3]:
nltk.download('brown')
nltk.download('universal_tagset')

[nltk_data] Downloading package brown to /home/jupyter/nltk_data...
[nltk_data]   Package brown is already up-to-date!
[nltk_data] Downloading package universal_tagset to
[nltk_data]     /home/jupyter/nltk_data...
[nltk_data]   Package universal_tagset is already up-to-date!


True

In [4]:
corpus = brown.tagged_sents(tagset='universal')
corpus

[[('The', 'DET'), ('Fulton', 'NOUN'), ('County', 'NOUN'), ('Grand', 'ADJ'), ('Jury', 'NOUN'), ('said', 'VERB'), ('Friday', 'NOUN'), ('an', 'DET'), ('investigation', 'NOUN'), ('of', 'ADP'), ("Atlanta's", 'NOUN'), ('recent', 'ADJ'), ('primary', 'NOUN'), ('election', 'NOUN'), ('produced', 'VERB'), ('``', '.'), ('no', 'DET'), ('evidence', 'NOUN'), ("''", '.'), ('that', 'ADP'), ('any', 'DET'), ('irregularities', 'NOUN'), ('took', 'VERB'), ('place', 'NOUN'), ('.', '.')], [('The', 'DET'), ('jury', 'NOUN'), ('further', 'ADV'), ('said', 'VERB'), ('in', 'ADP'), ('term-end', 'NOUN'), ('presentments', 'NOUN'), ('that', 'ADP'), ('the', 'DET'), ('City', 'NOUN'), ('Executive', 'ADJ'), ('Committee', 'NOUN'), (',', '.'), ('which', 'DET'), ('had', 'VERB'), ('over-all', 'ADJ'), ('charge', 'NOUN'), ('of', 'ADP'), ('the', 'DET'), ('election', 'NOUN'), (',', '.'), ('``', '.'), ('deserves', 'VERB'), ('the', 'DET'), ('praise', 'NOUN'), ('and', 'CONJ'), ('thanks', 'NOUN'), ('of', 'ADP'), ('the', 'DET'), ('City

In [5]:
inputs = []
targets = []

for sentence_tag_pairs in corpus:
    tokens = []
    target = []
    for token, tag in sentence_tag_pairs:
        tokens.append(token)
        target.append(tag)
    inputs.append(tokens)
    targets.append(target)

In [6]:
# save data to json format
with open('data.json', 'w') as f:
    for x, y in zip(inputs, targets):
        j = {'inputs': x, 'targets': y}
        s = json.dumps(j)
        f.write(f"{s}\n")

In [7]:
data = load_dataset("json", data_files='data.json')

Generating train split: 0 examples [00:00, ? examples/s]

In [8]:
data

DatasetDict({
    train: Dataset({
        features: ['inputs', 'targets'],
        num_rows: 57340
    })
})

In [9]:
small = data["train"].shuffle(seed=42).select(range(20_000))
small

Dataset({
    features: ['inputs', 'targets'],
    num_rows: 20000
})

In [10]:
data = small.train_test_split(seed=42)

In [11]:
data["train"][0]

{'inputs': ['Ulyate',
  'and',
  'Kearton',
  'climbed',
  'on',
  'toward',
  'the',
  'sound',
  'of',
  'the',
  'barking',
  'of',
  'the',
  'dogs',
  'and',
  'the',
  'sporadic',
  'roaring',
  'of',
  'the',
  'lion',
  ',',
  'till',
  'they',
  'came',
  ',',
  'out',
  'of',
  'breath',
  ',',
  'to',
  'the',
  'crest',
  ',',
  'and',
  'peering',
  'through',
  'the',
  'branches',
  'of',
  'a',
  'bush',
  ',',
  'this',
  'is',
  'what',
  'Ulyate',
  'saw',
  ':',
  'Jones',
  'who',
  'had',
  'apparently',
  '(',
  'and',
  'actually',
  'had',
  ')',
  'ridden',
  'up',
  'the',
  'nearly',
  'impassable',
  'hillside',
  ',',
  'sitting',
  'calmly',
  'on',
  'his',
  'horse',
  'within',
  'forty',
  'feet',
  'of',
  'a',
  'full-grown',
  'young',
  'lioness',
  ',',
  'who',
  'was',
  'crouched',
  'on',
  'a',
  'flat',
  'rock',
  'and',
  'seemed',
  'just',
  'about',
  'to',
  'charge',
  'him',
  ',',
  'while',
  'the',
  'dogs',
  'whirled',
  'aroun

In [12]:
data["train"].features

{'inputs': Sequence(feature=Value(dtype='string', id=None), length=-1, id=None),
 'targets': Sequence(feature=Value(dtype='string', id=None), length=-1, id=None)}

In [13]:
# map targets to ints
target_set = set()
for target in targets:
    target_set = target_set.union(target)
target_set

{'.',
 'ADJ',
 'ADP',
 'ADV',
 'CONJ',
 'DET',
 'NOUN',
 'NUM',
 'PRON',
 'PRT',
 'VERB',
 'X'}

In [14]:
target_list = list(target_set)
id2label = {k: v for k, v in enumerate(target_list)}
label2id = {v: k for k, v in id2label.items()}

## Data Tokenization

In [15]:
# We could also try using BERT directly
checkpoint = "distilbert-base-cased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

If we want to tokenize a doc that has been already split into words, like this one, we just need to pass in the argument `is_split_into_words=True`:

In [16]:
idx = 0
t = tokenizer(data['train'][idx]['inputs'], is_split_into_words=True)
t

{'input_ids': [101, 158, 25928, 1566, 1105, 26835, 9349, 1320, 5998, 1113, 1755, 1103, 1839, 1104, 1103, 26635, 1104, 1103, 6363, 1105, 1103, 188, 27695, 23041, 1104, 1103, 11160, 117, 6174, 1152, 1338, 117, 1149, 1104, 2184, 117, 1106, 1103, 13468, 117, 1105, 19205, 1194, 1103, 5020, 1104, 170, 13771, 117, 1142, 1110, 1184, 158, 25928, 1566, 1486, 131, 2690, 1150, 1125, 4547, 113, 1105, 2140, 1125, 114, 17698, 1146, 1103, 2212, 24034, 11192, 1895, 25068, 117, 2807, 13285, 1113, 1117, 3241, 1439, 5808, 1623, 1104, 170, 1554, 118, 4215, 1685, 11160, 5800, 117, 1150, 1108, 15062, 1113, 170, 3596, 2067, 1105, 1882, 1198, 1164, 1106, 2965, 1140, 117, 1229, 1103, 6363, 18370, 1213, 1123, 119, 102], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,

In [17]:
type(t)

transformers.tokenization_utils_base.BatchEncoding

We see that although the tokenized data seemed like a dictionary, it's really a BatchEncoding object. This object has some useful object methods we can use, e.g. we can call tokens() to see the tokens represented in string format, instead of ints.

In [18]:
t.tokens() 

['[CLS]',
 'U',
 '##lya',
 '##te',
 'and',
 'Ke',
 '##art',
 '##on',
 'climbed',
 'on',
 'toward',
 'the',
 'sound',
 'of',
 'the',
 'barking',
 'of',
 'the',
 'dogs',
 'and',
 'the',
 's',
 '##poradic',
 'roaring',
 'of',
 'the',
 'lion',
 ',',
 'till',
 'they',
 'came',
 ',',
 'out',
 'of',
 'breath',
 ',',
 'to',
 'the',
 'crest',
 ',',
 'and',
 'peering',
 'through',
 'the',
 'branches',
 'of',
 'a',
 'bush',
 ',',
 'this',
 'is',
 'what',
 'U',
 '##lya',
 '##te',
 'saw',
 ':',
 'Jones',
 'who',
 'had',
 'apparently',
 '(',
 'and',
 'actually',
 'had',
 ')',
 'ridden',
 'up',
 'the',
 'nearly',
 'imp',
 '##ass',
 '##able',
 'hillside',
 ',',
 'sitting',
 'calmly',
 'on',
 'his',
 'horse',
 'within',
 'forty',
 'feet',
 'of',
 'a',
 'full',
 '-',
 'grown',
 'young',
 'lion',
 '##ess',
 ',',
 'who',
 'was',
 'crouched',
 'on',
 'a',
 'flat',
 'rock',
 'and',
 'seemed',
 'just',
 'about',
 'to',
 'charge',
 'him',
 ',',
 'while',
 'the',
 'dogs',
 'whirled',
 'around',
 'her',
 '.',
 

We see the usual `[CLS]` and `[SEP]` [special tokens from BERT](https://www.neclab.eu/human-centric-ai-a-road-map-to-human-ai-collaboration/attending-to-future-tokens-for-bidirectional-sequence-generation#:~:text=The%20special%20tokens%20have%20specific,with%20the%20%5BMASK%5D%20token). Challenge: We have an input sequence and an output sequence, and their lengths vary. And we need to align the targets to the tokens, as tokens (words) have been split into sub-tokens (sub-words), but targets in NER are at the token (word) level.

To fix this, we will **expand** our dataset, in the sense that for any word split into multiple tokens, we’ll assign the same target.

But wait! What about special tokens from BERT (e.g. [CLS] and [SEP])? We need to create targets for those too.
- Why do we need to do this? Transformers work that way. It’s similar to the RNN concept for the same task, we have (x(t)  h(t)  y(t), for all t), so we need all that tokens representation in their former and latter stage.
- What value should we set for them? -100. Why? Because that’s the value that Hugging Face uses to know it needs to ignore them, not using them to update the model weights.

We’ll need to write our own algorithm for this.

In [19]:
# value of i indicates it's the i'th word in the input sentence (counting from 0)
t.word_ids()

[None,
 0,
 0,
 0,
 1,
 2,
 2,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 46,
 46,
 47,
 48,
 49,
 50,
 51,
 52,
 53,
 54,
 55,
 56,
 57,
 58,
 59,
 60,
 61,
 62,
 62,
 62,
 63,
 64,
 65,
 66,
 67,
 68,
 69,
 70,
 71,
 72,
 73,
 74,
 75,
 75,
 75,
 76,
 77,
 77,
 78,
 79,
 80,
 81,
 82,
 83,
 84,
 85,
 86,
 87,
 88,
 89,
 90,
 91,
 92,
 93,
 94,
 95,
 96,
 97,
 98,
 99,
 100,
 None]

In [20]:
def align_targets(labels, word_ids):
    aligned_labels = []
    for word in word_ids:
        if word is None:         # token like [CLS]
            label = -100
        else:                    # it's a real word
            label = label2id[labels[word]]

        # add the label 
        aligned_labels.append(label)

    return aligned_labels

In [21]:
# try our function
labels = data['train'][idx]['targets']
word_ids = t.word_ids()
aligned_targets = align_targets(labels, word_ids)
aligned_targets

[-100,
 4,
 4,
 4,
 10,
 4,
 4,
 4,
 2,
 0,
 11,
 7,
 4,
 11,
 7,
 4,
 11,
 7,
 4,
 10,
 7,
 1,
 1,
 4,
 11,
 7,
 4,
 5,
 11,
 9,
 2,
 5,
 0,
 11,
 4,
 5,
 11,
 7,
 4,
 5,
 10,
 2,
 11,
 7,
 4,
 11,
 7,
 4,
 5,
 7,
 2,
 7,
 4,
 4,
 4,
 2,
 5,
 4,
 9,
 2,
 6,
 5,
 10,
 6,
 2,
 5,
 2,
 11,
 7,
 6,
 1,
 1,
 1,
 4,
 5,
 2,
 6,
 11,
 7,
 4,
 11,
 8,
 4,
 11,
 7,
 1,
 1,
 1,
 1,
 4,
 4,
 5,
 9,
 2,
 2,
 11,
 7,
 1,
 4,
 10,
 2,
 6,
 6,
 0,
 2,
 9,
 5,
 11,
 7,
 4,
 2,
 11,
 9,
 5,
 -100]

In [22]:
aligned_labels = [id2label[i] if i >= 0 else None for i in aligned_targets]
for x, y in zip(t.tokens(), aligned_labels):
    print(f"{x}\t{y}")

[CLS]	None
U	NOUN
##lya	NOUN
##te	NOUN
and	CONJ
Ke	NOUN
##art	NOUN
##on	NOUN
climbed	VERB
on	PRT
toward	ADP
the	DET
sound	NOUN
of	ADP
the	DET
barking	NOUN
of	ADP
the	DET
dogs	NOUN
and	CONJ
the	DET
s	ADJ
##poradic	ADJ
roaring	NOUN
of	ADP
the	DET
lion	NOUN
,	.
till	ADP
they	PRON
came	VERB
,	.
out	PRT
of	ADP
breath	NOUN
,	.
to	ADP
the	DET
crest	NOUN
,	.
and	CONJ
peering	VERB
through	ADP
the	DET
branches	NOUN
of	ADP
a	DET
bush	NOUN
,	.
this	DET
is	VERB
what	DET
U	NOUN
##lya	NOUN
##te	NOUN
saw	VERB
:	.
Jones	NOUN
who	PRON
had	VERB
apparently	ADV
(	.
and	CONJ
actually	ADV
had	VERB
)	.
ridden	VERB
up	ADP
the	DET
nearly	ADV
imp	ADJ
##ass	ADJ
##able	ADJ
hillside	NOUN
,	.
sitting	VERB
calmly	ADV
on	ADP
his	DET
horse	NOUN
within	ADP
forty	NUM
feet	NOUN
of	ADP
a	DET
full	ADJ
-	ADJ
grown	ADJ
young	ADJ
lion	NOUN
##ess	NOUN
,	.
who	PRON
was	VERB
crouched	VERB
on	ADP
a	DET
flat	ADJ
rock	NOUN
and	CONJ
seemed	VERB
just	ADV
about	ADV
to	PRT
charge	VERB
him	PRON
,	.
while	ADP
the	DET
dogs	NOUN
whirled	VER

In [23]:
# tokenize both inputs and targets
def tokenize_fn(batch):
    # tokenize the input sequence first
    # this populates input_ids, attention_mask, etc.
    tokenized_inputs = tokenizer(
        batch['inputs'], truncation=True, is_split_into_words=True
    )

    labels_batch = batch['targets'] # original targets
    aligned_labels_batch = []
    for i, labels in enumerate(labels_batch):
        word_ids = tokenized_inputs.word_ids(i)
        aligned_labels_batch.append(align_targets(labels, word_ids))

    # recall: the 'target' must be stored in key called 'labels'
    tokenized_inputs['labels'] = aligned_labels_batch

    return tokenized_inputs

The former function returns input our tokenized inputs, now containing their ids, attention masks, and so forth, but also the aligned labels.

In [24]:
# want to remove these from model inputs - they are neither inputs nor targets
data["train"].column_names

['inputs', 'targets']

In [25]:
tokenized_datasets = data.map(
  tokenize_fn,
  batched=True,
  remove_columns=data["train"].column_names,
)

Map:   0%|          | 0/15000 [00:00<?, ? examples/s]

Map:   0%|          | 0/5000 [00:00<?, ? examples/s]

In [26]:
tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 15000
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 5000
    })
})

As we see, columns in our dataset are the ones we need now.

### Data Collator

In [27]:
data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer)

In [28]:
# https://stackoverflow.com/questions/11264684/flatten-list-of-lists
def flatten(list_of_lists):
    flattened = [val for sublist in list_of_lists for val in sublist]
    return flattened

## Model evaluation

Unfortunately, `seqeval` just works for IOB format (which is for NER), so in this case we'll need to compute the metrics manually without using this library.

In [29]:
import numpy as np
from sklearn.metrics import f1_score, accuracy_score

def compute_metrics(logits_and_labels):
    logits, labels = logits_and_labels
    preds = np.argmax(logits, axis=-1)

    # remove -100 from labels and predictions
    labels_jagged = [[t for t in label if t != -100] for label in labels]

    # do the same for predictions whenever true label is -100
    preds_jagged = [[p for p, t in zip(ps, ts) if t != -100] \
        for ps, ts in zip(preds, labels)
    ]

    # flatten labels and preds
    labels_flat = flatten(labels_jagged)
    preds_flat = flatten(preds_jagged)

    acc = accuracy_score(labels_flat, preds_flat)
    f1 = f1_score(labels_flat, preds_flat, average='macro')

    return {
        'f1': f1,
        'accuracy': acc,
    }

In [30]:
labels = [[-100, 0, 0, 1, 2, 1, -100]]
logits = np.array([[
  [0.8, 0.1, 0.1],
  [0.8, 0.1, 0.1],
  [0.8, 0.1, 0.1],
  [0.1, 0.8, 0.1],
  [0.1, 0.8, 0.1],
  [0.1, 0.8, 0.1],
  [0.1, 0.8, 0.1],
]])

compute_metrics((logits, labels))

{'f1': 0.6, 'accuracy': 0.8}

In [31]:
model = AutoModelForTokenClassification.from_pretrained(
    checkpoint,
    id2label=id2label,
    label2id=label2id,
)

Some weights of DistilBertForTokenClassification were not initialized from the model checkpoint at distilbert-base-cased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


**Note that** we get metrics for each entity, and then overall metrics. 
**Note 2:** It seems off that ORG precision is 0, even if we did some correct predictions (it should intuitively be 0.5). This is because seqeval does some special computation for ER evaluation (so their evaluation differs from the generic ML one).

Let's now write our compute metrics function.

In [32]:
import numpy as np
from sklearn.metrics import f1_score, accuracy_score

def compute_metrics(logits_and_labels):
    logits, labels = logits_and_labels
    preds = np.argmax(logits, axis=-1)

    # remove -100 from labels and predictions
    labels_jagged = [[t for t in label if t != -100] for label in labels]

    # do the same for predictions whenever true label is -100
    preds_jagged = [[p for p, t in zip(ps, ts) if t != -100] \
        for ps, ts in zip(preds, labels)
    ]

    # flatten labels and preds
    labels_flat = flatten(labels_jagged)
    preds_flat = flatten(preds_jagged)

    acc = accuracy_score(labels_flat, preds_flat)
    f1 = f1_score(labels_flat, preds_flat, average='macro')

    return {
        'f1': f1,
        'accuracy': acc,
    }

In [33]:
labels = [[-100, 0, 0, 1, 2, 1, -100]]
logits = np.array([[
  [0.8, 0.1, 0.1],
  [0.8, 0.1, 0.1],
  [0.8, 0.1, 0.1],
  [0.1, 0.8, 0.1],
  [0.1, 0.8, 0.1],
  [0.1, 0.8, 0.1],
  [0.1, 0.8, 0.1],
]])
compute_metrics((logits, labels))

{'f1': 0.6, 'accuracy': 0.8}

## Model and trainer

We'll use AutoModelForTokenClassification, which is accurate for our current task. For this, we need to specify the labels in an alternative way, passing the arguments id2label and label2id, defined based on our input dataset.

In [34]:
from transformers import AutoModelForTokenClassification

model = AutoModelForTokenClassification.from_pretrained(
    checkpoint,
    id2label=id2label,
    label2id=label2id,
)

Some weights of DistilBertForTokenClassification were not initialized from the model checkpoint at distilbert-base-cased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [35]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    "distilbert-finetuned-ner",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    num_train_epochs=2,
)

In [36]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    tokenizer=tokenizer,
)
trainer.train()

You're using a DistilBertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,F1,Accuracy
1,0.070700,0.058065,0.946801,0.982971
2,0.028600,0.052729,0.956710,0.985548


TrainOutput(global_step=3750, training_loss=0.0669231357574463, metrics={'train_runtime': 233.4162, 'train_samples_per_second': 128.526, 'train_steps_per_second': 16.066, 'total_flos': 387922021634304.0, 'train_loss': 0.0669231357574463, 'epoch': 2.0})

In [37]:
trainer.save_model('my_saved_model')

In [38]:
from transformers import pipeline

pipe = pipeline(
  "token-classification",
  model='my_saved_model',
  device=0,
)

In [39]:
s = "Bill Gates was the CEO of Microsoft in Seattle, Washington."
pipe(s)

[{'entity': 'NOUN',
  'score': 0.9996592,
  'index': 1,
  'word': 'Bill',
  'start': 0,
  'end': 4},
 {'entity': 'NOUN',
  'score': 0.9997565,
  'index': 2,
  'word': 'Gates',
  'start': 5,
  'end': 10},
 {'entity': 'VERB',
  'score': 0.9997701,
  'index': 3,
  'word': 'was',
  'start': 11,
  'end': 14},
 {'entity': 'DET',
  'score': 0.9998252,
  'index': 4,
  'word': 'the',
  'start': 15,
  'end': 18},
 {'entity': 'NOUN',
  'score': 0.9996026,
  'index': 5,
  'word': 'CEO',
  'start': 19,
  'end': 22},
 {'entity': 'ADP',
  'score': 0.9998227,
  'index': 6,
  'word': 'of',
  'start': 23,
  'end': 25},
 {'entity': 'NOUN',
  'score': 0.9993993,
  'index': 7,
  'word': 'Microsoft',
  'start': 26,
  'end': 35},
 {'entity': 'ADP',
  'score': 0.9997732,
  'index': 8,
  'word': 'in',
  'start': 36,
  'end': 38},
 {'entity': 'NOUN',
  'score': 0.9998217,
  'index': 9,
  'word': 'Seattle',
  'start': 39,
  'end': 46},
 {'entity': '.',
  'score': 0.99977714,
  'index': 10,
  'word': ',',
  'star